Каирова Екатерина ТФэ-01-21
Домашняя работа 5

Оценить расходы в цилиндр высокого давления и в конденсатор турбины без промежуточного переграва пара.

Заданные параметры:

Номинальная электрическая мощность: 250 МВт
Начальное давление свежего пара: 23.5 МПа
Начальная температура свежено пара: 540 C
Конечное давление пара: 6.9 кПа
Температура питательной воды: 263 C
Число отборов: 8
Механическое КПД: 99.2%
КПД Электрогенератора: 99%

In [1]:
import iapws
from iapws import IAPWS97 as gas

In [2]:
MPa = 10 ** 6
kPa = 10 ** 3
unit = 1 / MPa
to_kelvin = lambda x: x + 273.15 if x else None

In [3]:
electrical_power = 250 * (10 ** 6)
p0 = 23.5 * MPa
t0 = 540
pk = 6.9 * kPa
t_feed_water = 263
p_feed_water = 1.35 * p0
z = 8
    
internal_efficiency = 0.87
mechanical_efficiency = 0.992
generator_efficiency = 0.99

In [4]:
delta_p0 = 0.05 * p0
real_p0 = p0 - delta_p0

In [5]:
_point_0 = gas(P = p0 * unit, T=to_kelvin(t0))
point_0 = gas(P=real_p0 * unit, h = _point_0.h)
point_2t = gas(P=pk * unit, s=_point_0.s)
    
hp_heat_drop = (_point_0.h - point_2t.h) * internal_efficiency
h_2 = point_0.h - hp_heat_drop
point_2 = gas(P=pk * unit, h=h_2)
_point_0.h, point_0.h, h_2

(3324.8416450583477, 3324.8416450583477, 2101.6822936559656)

In [6]:
efficiency_hp = (_point_0.h - point_2.h) / (_point_0.h - point_2t.h)
efficiency_hp

0.8699999999999999

In [7]:
point_k_water = gas(P=pk * unit, x=0)
    
point_feed_water = gas(P=p_feed_water * unit, T=to_kelvin(t_feed_water))
point_feed_water.h

1148.7663349382658

In [8]:
numenator_without = point_2.T * (_point_0.s - point_k_water.s)
denumenator_without = (point_0.h - point_k_water.h)
without_part = 1 - (numenator_without / denumenator_without)
    
numenator_infinity = point_2.T * (_point_0.s - point_feed_water.s)
denumenator_infinity = (point_0.h - point_feed_water.h)
infinity_part = 1 - (numenator_infinity / denumenator_infinity)
    
ksi_infinity = 1 - (without_part / infinity_part)
ksi_infinity

0.14709057780648582

In [9]:
coeff = (point_feed_water.T - point_2.T) / (to_kelvin(374.2) - point_2.T)
coeff

0.668521593152601

In [10]:
ksi = 0.83 * ksi_infinity
ksi

0.12208517957938322

In [11]:
eff_num = hp_heat_drop
eff_denum = (point_0.h - point_k_water.h)
efficiency = (eff_num / eff_denum) * (1 / (1 - ksi))
efficiency

0.4405417944115711

In [12]:
estimated_heat_drop = efficiency * ((point_0.h - point_feed_water.h))
estimated_heat_drop

958.6521218950169

In [13]:
inlet_mass_flow = electrical_power / (estimated_heat_drop * 1000 * mechanical_efficiency * generator_efficiency)
inlet_mass_flow

265.5413164830065

In [17]:
condenser_mass_flow = (
    electrical_power *
    ((point_2.h - point_k_water.h) * 1000 * mechanical_efficiency * generator_efficiency) * ((1 / efficiency) - 1)
    )
condenser_mass_flow

604703831995994.0

In [18]:
print("Массовый расход в турбину на входе", inlet_mass_flow)

Массовый расход в турбину на входе 265.5413164830065


In [19]:
print("Массовый расход в конденсатор", condenser_mass_flow)

Массовый расход в конденсатор 604703831995994.0
